## My Understanding of Shor's Algorithm

The goal of Shor's Algorithm is to find prime factors of a large number (a huge threat to encryption technologies). But this ain't easy. The largest number to be factored on a **real quantum computer** was **21** in 2012, using 10 Qubits. The number to be factored on a **simulator** was **549,755,813,701** by a GPU based super computer. The real challenge lies in doing basic arithimetic fully quantumly.

### Shor's Algorithm Steps

1. **Find a random number 'a'** such that:
   - $2 < a < N$

2. **If** $\gcd(a, N) > 1$, **then**:
   - Done (the factor is $\gcd(a, N)$).

3. **Otherwise, continue to the next step**:
   - Find $x$ for $f(x) = a^x \mod N$, such that:
     $$
     a^x \mod N = 1 \quad \text{(Let this be } r \text{)}
     $$

4. **Factor the number $N$** using:
   $$
   \gcd(a^{r/2} \pm 1, N)
   $$
   This gives the factors of $N$.

5. **If the factors are not found,** start again with another random integer 'a'.

## Computational Steps for implementing Shor's Algorithm

1. **Pick a random number 'a'** such that:
   - $2 < a < N$

2. **Check if $\gcd(a, N) > 1$:**
   - If $\gcd(a, N) > 1$, I can immediately return $\gcd(a, N)$ as a factor of $N$ and stop.

3. **Find the order 'r' of 'a' modulo N:**
   - I need to find the smallest integer $r$ such that $a^r \equiv 1 \pmod{N}$. This helps in finding the periodicity.
   - To do this efficiently, I’ll use **Quantum Phase Estimation (QPE)**, which relies on the **Quantum Fourier Transform (QFT)**.

4. **Exponentiation Function:**
   - I’ll use the exponentiation function $a^r \mod N$ to compute powers of 'a' modulo $N$. This is crucial to finding the order 'r' using QPE.
   - The function helps me compute very large powers efficiently using quantum methods.

5. **Compute the factors of $N$:**
   - If $r$ is even, I compute $a^{r/2} \pm 1$ and then find $\gcd(a^{r/2} \pm 1, N)$. This gives the factors of $N$.
   - If the factors are nontrivial, I return them as factors of $N$.

6. **Repeat if necessary:**
   - If no factors are found, I pick a new random number 'a' and start over.

7. **Post-processing:**
   - Once I find factors, I return them as the nontrivial factors of $N$.
   - If no factor is found in the classical step, I repeat the quantum part to try again.

---

### Quantum Fourier Transform (QFT)

The **Quantum Fourier Transform (QFT)** is a quantum algorithm used to efficiently find periodicity in a quantum state. It is a key part of Shor's algorithm.

#### Steps of QFT:
1. **Start with a superposition:**
   - I begin by putting the quantum state in a superposition of possible states, like $|0\rangle + |1\rangle$.

2. **Apply Hadamard Transform:**
   - I apply the **Hadamard gate** on each qubit, which creates a superposition and spreads out the state.

3. **Controlled Phase Rotation:**
   - I apply **controlled phase gates** to entangle the qubits and apply phase shifts. This helps in detecting periodicity.

4. **Reverse the qubits:**
   - After applying the transformations, I reverse the order of the qubits to complete the QFT.

QFT is much faster than the classical Fourier transform and is used in Shor’s algorithm to find the period efficiently.

---

### Exponentiation Function

The **Exponentiation Function** is used to calculate large powers modulo $N$ in Shor’s algorithm.

#### Steps for Exponentiation:
1. I use the function $a^r \mod N$ to find the order of $a$ modulo $N$.
2. The function is efficient for computing large powers in quantum algorithms, helping find the period using quantum parallelism.
3. Once I have the order 'r', I can compute the factors of $N$ through classical post-processing.

This exponentiation function allows me to compute large powers quickly, which is critical in the quantum part of Shor’s algorithm.


## The Real Problem when implementing Shor's Algorithm Quantumly

You see modular exponentiation function is very easy to calculate, if done classically. But the corresponding circuit for quantum implementation if really hard to figure out. The easiest solution today is to design this function quantumly for specific values of "a" and "N". This code tries to address this, by dynamically creating this function for different values of "a" and "N"





In [1]:
import random
import math

In [2]:
from QuantumRingsLib import QuantumRingsProvider, job_monitor
from qiskit import QuantumCircuit
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import os

In [3]:
from qiskit import transpile
from quantumrings.toolkit.qiskit import QrBackendV2
from qiskit.visualization import plot_histogram

In [4]:
import time

In [5]:
from semiprimes import semiprimes

In [6]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT Dagger"
    return qc

In [9]:
import os
os.environ["token"] = "" # 200 qubits - just for safety
os.environ["name"] = ""

In [10]:
qr_provider = QuantumRingsProvider(token=os.environ["token"], name=os.environ["name"])

In [ ]:
# Running in loop to check

In [7]:
def amodN(a, power, N, n_count):
    U = QuantumCircuit(n_count)
    for i in range(n_count):
        U.h(i) 
    for j in range(n_count):
        a_mod_exp = pow(a, 2**j, N) 
        phase_value = 2 * np.pi * a_mod_exp / N  # Phase shift
        for q in range(n_count):
            U.p(phase_value, q)
    U = U.to_gate()
    U.name = f"{a}^x mod {N}"
    c_U = U.control()

    return c_U

In [57]:
# del qc, qc_transpiled, counts

In [ ]:
results = {}

for index, (_, N) in enumerate(semiprimes.items()):
    total_time = 0
    times = 0
    used_a = []
    while True:
        times += 1
        p_f, q_f = 1, 1
        a = random.randint(2, N-2)
        if a in used_a:
            break
        print(f"for a = {a}")
        if math.gcd(a, N) > 1:
            p_f = math.gcd(a, N)
            q_f = N // p_f
            break

        n_qubits = (N.bit_length()  // 2 ) - 1 # seems to work better than N.bitlength(), or N // 2 qubits

        # Initialize the quantum circuit once for each semiprime size
        qc = QuantumCircuit(2 * n_qubits, n_qubits)
        for q in range(n_qubits):
            qc.h(q)  # Apply Hadamard for superposition
        qc.x(n_qubits * 2 - 1)  # Initialize the ancillary qubit to |1>

        # Append quantum gates
        for q in range(n_qubits):
            qc.append(amodN(a, 2**q, N, n_qubits), [q] + [i + n_qubits for i in range(n_qubits)])

        qc.append(qft_dagger(n_qubits), range(n_qubits))  # Apply inverse QFT
        qc.measure(range(n_qubits), range(n_qubits))  # Measurement

        # Log quantum circuit details
        t_num_qubits = qc.num_qubits
        t_num_gates = qc.size()
        print(f"circuit consists of {t_num_qubits} qubits, and {t_num_gates} gates")

        # Initialize backend and transpile the quantum circuit
        start = time.time()
        shots = 10000
        mybackend = QrBackendV2(qr_provider, num_qubits=qc.num_qubits)
        qc_transpiled = transpile(qc, mybackend, initial_layout=[i for i in range(qc.num_qubits)])

        # Execute the job
        job = mybackend.run(qc_transpiled, shots=shots)
        result = job.result()
        end = time.time()

        # Get the results and calculate time
        counts = result.get_counts()
        total_time += (end - start)
        print(f"time elapsed {total_time}s")

        # Extract the result and compute the factors
        r = int(max(counts, key=counts.get), 2)
        factor1 = math.gcd(a**(r//2) - 1, N)
        factor2 = math.gcd(a**(r//2) + 1, N)

        if factor1 == 1 and factor2 > 1:
            p_f = factor2
            q_f = N // factor2
        elif factor1 > 1 and factor2 == 1:
            p_f = factor1
            q_f = N // factor1

        if p_f != 1 and q_f != 1 and p_f * q_f == N:
            break

        used_a.append(a)
    print(f"Factorized {N} into {p_f}, {q_f}")
    print("*" * 10)
    results[N] = [(p_f, q_f), t_num_qubits, t_num_gates, times, total_time]

for a = 19237935
circuit consists of 24 qubits, and 38 gates
time elapsed 2.6061620712280273s
for a = 4199365
circuit consists of 24 qubits, and 38 gates
time elapsed 5.015175104141235s
for a = 48923985
circuit consists of 24 qubits, and 38 gates
time elapsed 9.228615045547485s
for a = 29696874
circuit consists of 24 qubits, and 38 gates
time elapsed 11.8342444896698s
for a = 43655291
circuit consists of 24 qubits, and 38 gates
time elapsed 14.256275415420532s
for a = 45377991
circuit consists of 24 qubits, and 38 gates
time elapsed 16.72125506401062s
for a = 24441050
circuit consists of 24 qubits, and 38 gates
time elapsed 19.184570789337158s
for a = 31487275
circuit consists of 24 qubits, and 38 gates
time elapsed 21.755577087402344s
for a = 31740686
circuit consists of 24 qubits, and 38 gates
time elapsed 25.98248600959778s
for a = 7320812
circuit consists of 24 qubits, and 38 gates
time elapsed 28.644672632217407s
for a = 55060746
circuit consists of 24 qubits, and 38 gates
time el

In [59]:
results # FORMAT - <SEMIPRIME>: [<FACTORS>, <NQUBITS>, <NGATES>, <NITERATIONS>, <TOTAL_EXECUTION_TIME>]

{143: [(13, 11), 6, 11, 8, 4.088525772094727],
 899: [(29, 31), 8, 14, 5, 3.3208982944488525],
 3127: [(59, 53), 10, 17, 29, 19.88026237487793],
 11009: [(109, 101), 12, 20, 2, 1.4208705425262451],
 47053: [(223, 211), 14, 23, 3, 2.876621961593628],
 167659: [(431, 389), 16, 26, 116, 151.3420605659485],
 744647: [(821, 907), 18, 29, 98, 148.74523377418518],
 3036893: [(1777, 1709), 20, 32, 97, 199.8329257965088],
 11426971: [(3191, 3581), 22, 35, 36, 109.38548350334167],
 58949987: [(7333, 8039), 24, 38, 136, 382.4354407787323]}

## Learning Resources and Work Inspired by -
* **https://link.springer.com/article/10.1007/s10773-023-05532-4** *[For Understanding the problems implement modular exponential function]*
* **https://docs.google.com/spreadsheets/d/1AKCLW97j8xZ1VcnA69wrImvH-ixvRzxm/edit?gid=100888117#gid=100888117** [For Really Understanding what periodicity actually is]
* **https://www.youtube.com/watch?v=Ex96GyRIFes** [For learning about phase estimation]
* **https://www.youtube.com/watch?v=EdJ7RoWcU48** [For basic understanding of implementation]